# Charity Funding Predictor
#### Using machine learning and neural networks, predict whether applicants will be successful if funded by Alphabet Soup

#### Charity dataset that capture metadata about each organization:
* **EIN** and **NAME**—Identification columns
* **APPLICATION_TYPE**—Alphabet Soup application type
* **AFFILIATION**—Affiliated sector of industry
* **CLASSIFICATION**—Government organization classification
* **USE_CASE**—Use case for funding
* **ORGANIZATION**—Organization type
* **STATUS**—Active status
* **INCOME_AMT**—Income classification
* **SPECIAL_CONSIDERATIONS**—Special consideration for application
* **ASK_AMT**—Funding amount requested
* **IS_SUCCESSFUL**—Was the money used effectively

## Step 1: Preprocess the Data

In [1]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
# application_df = pd.read_csv("Resources/charity_data.csv")
application_df = pd.read_csv("/content/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# this is the target columns
application_df['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

* What variable(s) are the target(s) for your model?
* What variable(s) are the feature(s) for your model?

### Target variable: IS_SUCCESSFUL; 1 is successful, 0 (unsuccessful)
- IS_SUCCESSFUL              int64

### Feature variables:
- APPLICATION_TYPE          object
- AFFILIATION               object
- CLASSIFICATION            object
- USE_CASE                  object
- ORGANIZATION              object
- STATUS                     int64
- INCOME_AMT                object
- SPECIAL_CONSIDERATIONS    object
- ASK_AMT                    int64


In [5]:
application_df.shape

(34299, 12)

In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [8]:
# Determine the number of unique values in each column.
ctn_unique_values = application_df.nunique()
print(f'Number of unique values in each column : \n{ctn_unique_values}')

Number of unique values in each column : 
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [9]:
# Look at APPLICATION_TYPE value counts for binning
cnt_application_type = application_df['APPLICATION_TYPE'].value_counts()
cnt_application_type[cnt_application_type > 10]

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(cnt_application_type[cnt_application_type < 10].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
cnt_classification = application_df['CLASSIFICATION'].value_counts()
cnt_classification[cnt_classification > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(cnt_classification[cnt_classification < 10].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_application_df = pd.get_dummies(application_df)
numeric_application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = numeric_application_df["IS_SUCCESSFUL"] 

# Features columns are eveything without the target column
X = numeric_application_df.drop(["IS_SUCCESSFUL"],axis=1) 

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model(hidden_nodes_layer1, hidden_nodes_layer2):
    
    number_input_features = len(X_train_scaled[0])
    
    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(
          tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
    )

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    nn.summary()
    return nn

## Step 2: Compile, Train, and Evaluate the Model

### Attemp 1: Execute, Compile, Train, and Evaluate the Model

In [17]:
# Attemp 1: Execute the model
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
nn = create_model(hidden_nodes_layer1, 
                  hidden_nodes_layer2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 576       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 627
Trainable params: 627
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6306 - accuracy: 0.6678
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5694 - accuracy: 0.7253
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5623 - accuracy: 0.7300
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5578 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7312
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7318
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5501 - accuracy: 0.7286 - 520ms/epoch - 2ms/step
Loss: 0.5500602722167969, Accuracy: 0.7286297082901001


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

# Step 3: Optimize the Model
### Execute the same model by using different hidden nodes layer 

### Attemp 2: Execute, Compile, Train, and Evaluate the Model

In [22]:
# Execute the model
hidden_nodes_layer1 = 16
hidden_nodes_layer2 = 10
nn = create_model(hidden_nodes_layer1, 
                  hidden_nodes_layer2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                1152      
                                                                 
 dense_4 (Dense)             (None, 10)                170       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7324
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7327
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7333
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7343
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7347
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5452 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5534 - accuracy: 0.7272 - 466ms/epoch - 2ms/step
Loss: 0.55335932970047, Accuracy: 0.7272303104400635


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_2.h5")

### Attemp 3: Execute, Compile, Train, and Evaluate the Model

In [28]:
# Attemp 3 execute the model
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 20
nn = create_model(hidden_nodes_layer1,hidden_nodes_layer2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                2304      
                                                                 
 dense_7 (Dense)             (None, 20)                660       
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,985
Trainable params: 2,985
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5816 - accuracy: 0.7145
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7322
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7328
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5469 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accura

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5547 - accuracy: 0.7261 - 483ms/epoch - 2ms/step
Loss: 0.5546638369560242, Accuracy: 0.726064145565033


In [32]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_3.h5")